# Walkthrough - Gamma n Mutual Infor



In [1]:
import sys, os

# Insert path to model directory,.
cwd = os.getcwd()
path = f"{cwd}/../../src"
sys.path.insert(0, path)

# Insert path to package,.
pysim_path = f"/home/emmanuel/code/pysim/"
sys.path.insert(0, pysim_path)

import warnings
from typing import Optional, Tuple
import tqdm
import random
import pandas as pd
import numpy as np
import argparse
from sklearn.utils import check_random_state

# toy datasets
from data.distribution import DataParams, Inputs

# Kernel Dependency measure
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process.kernels import RBF
from models.dependence import HSICModel
from pysim.kernel.utils import estimate_sigma

# RBIG IT measures
# from models.ite_algorithms import run_rbig_models

# Plotting
from visualization.distribution import plot_scorer

# experiment helpers
from experiments.utils import dict_product, run_parallel_step
from tqdm import tqdm

# Plotting Procedures
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use(['seaborn-talk'])
warnings.filterwarnings('ignore') # get rid of annoying warnings
%matplotlib inline

warnings.filterwarnings('ignore') # get rid of annoying warnings

%load_ext autoreload
%autoreload 2

In [2]:
!pwd

/home/emmanuel/projects/2019_hsic_align/notebooks/4_distributions


In [3]:
FIG_PATH = "/home/emmanuel/projects/2019_hsic_align/results/figures/distribution_experiment/mutual_info/"
RES_PATH = "/home/emmanuel/projects/2019_hsic_align/data/results/distributions/mutual_info/"

### Experimental Parameters

In [4]:
# initialize the holder for the parameters
parameters = {}

#### Case I - HSIC Estimator

In this first part, we have 3 cases of HSIC as a combination of a centered kernel and whether or not we normalize the covariance term. The 3 "scorers" are as follows:


1. **HSIC**

$$HSIC = \frac{1}{n(n-1)}\langle K_xH,K_yH \rangle_F$$

> In this case, the kernels are **centered**, but the score is **not normalized**.


2. **Kernel Alignment** (KA) 

$$TKA = \frac{\langle K_x,K_y \rangle_F}{||K_x||_F||K_y||_F}$$

> In this case, the kernels are **not centered** but the score is **normalized**.

3. **cTKA**

$$cTKA = \frac{\langle K_xH,K_yH \rangle_F}{||K_xH||_F||K_yH||_F}$$

> In this case, the kernels are **centered** and the score is **normalized**.

In [5]:
def get_hsic(
    X: np.ndarray, 
    Y: np.ndarray, 
    scorer: str, 
    sigma_X: Optional[float]=None, 
    sigma_Y: Optional[float]=None
) -> float:
    """Estimates the HSIC value given some data, sigma and
    the score."""
    # init hsic model class
    
    hsic_model = HSICModel()
    # hsic model params
    if sigma_X is not None:
        
        hsic_model.kernel_X = RBF(sigma_X)
        hsic_model.kernel_Y = RBF(sigma_Y)

    # get hsic score
    hsic_val = hsic_model.get_score(X, Y, scorer)
    
    return hsic_val

# parameters
parameters['scorer'] = ['hsic', 'ka', 'cka'] 

#### Case II - Sigma Estimator

For this parameter, we are interested in estimating a few things:

1. We want to know which estimator to choose from.

Kernel methods are great if the parameters of the kernel are correct. In supervised scenarios, we can simply learn the appropriate kernel parameters that best fit our data given some criteria. In unsupervised settings, we generally do not know which parameters to choose from. But there are many different ways to choose the parameters as every lab/researcher has their own method that "they swear by". I will choose some of the most common ones: 

* Silverman
* Scott
* Mean Distance
* Median Distance
* Median Distance with the $k^{th}$ sample (or percent) of that distance matrix.

#### Case III - Sigma Application

2. We want to know the how we are applying the length scale.

We have three cases to consider:

* One length scale for both datasets
* One length scale per dataset
* One length scale per dataset per dimension

This is important as it could turn a good estimator into a bad estimator. Scott and Silverman work very well for univariate distributions but not very well for multivariate distributions. So if we have one scott/silverman estimate per feature, then this estimator might be a lot better and yield much better results. For the case of the RBF kernel, having one length scale per dimension corresponds to the ARD Kernel which assigns a length scale (or relevance values) per feature. We don't typically use the ARD kernel for kernel methods that we cannot optimize using some gradient function due to how expensive it is. But in this case, it isn't so expensive because we are choosing not to optimizing anything.

In [6]:
def get_sigma(
    X: np.ndarray, 
    Y: np.ndarray, 
    method: str='silverman', 
    percent: Optional[float]=None,
    per_dimension: bool=False, 
    separate_scales: bool=False
) -> Tuple[np.ndarray, np.ndarray]:
    # sigma parameters
    subsample = None
    random_state = 123
    
    sigma_X = estimate_sigma(
        X, 
        subsample=subsample,
        method=method,
        percent=percent,
        random_state=random_state,
        per_dimension=per_dimension
    )
    
    sigma_Y = estimate_sigma(
        Y, 
        subsample=subsample,
        method=method,
        percent=percent,
        random_state=random_state,
        per_dimension=per_dimension
    )
    
    if separate_scales is False:
        sigma_Y = sigma_X = np.mean([sigma_X, sigma_Y])
    
    return sigma_X, sigma_Y

# Parameters for the estimators
parameters['sigma_estimator'] = [
#     ('silverman',None),
#     ('scott', None),
    *[('median', x) for x in np.arange(0.1, 1.0, 0.1, dtype=np.float64)]
]
parameters['separate_scales'] = [True, False]
parameters['per_dimension'] = [True, False]

#### Case IV - Standardize or not Standardize

This is a simple case but it can have drastic changes in the results of estimating the length scale. In ML, we tend to standardize our datasets because the algorithms do better with predictions with the ranges are contained. Datasets with massive values for certain features could have adverse affects on the representation and the predictions. The formula is given by:

$$\bar{x} = \frac{x - \mu_x}{\sigma_x}$$

**Note**: this is scaling per feature and not per sample. 

In [7]:
[('median', i) for i in np.arange(0.1, 1.0, 0.1, dtype=np.float64)]

[('median', 0.1),
 ('median', 0.2),
 ('median', 0.30000000000000004),
 ('median', 0.4),
 ('median', 0.5),
 ('median', 0.6),
 ('median', 0.7000000000000001),
 ('median', 0.8),
 ('median', 0.9)]

In [8]:
# from typing import Tuple, Optional

def standardize_data(
    X: np.ndarray, 
    Y: np.ndarray, 
    standardize: bool=False
) -> Tuple[np.ndarray, np.ndarray]:
    X = StandardScaler().fit_transform(X)
    Y = StandardScaler().fit_transform(Y)
    return X, Y

# experimental parameters
parameters['standardize'] = [True, False]

### Case V - Multivariate Datasets

For this experiment, we have generated samples for two sets of multivariate distributions: the Gaussian and the T-Student. We have varied the parameters so that we get a variety of samples, dimensions and the amount of similarity (that we can analytically calculate) between them. 

For example, we can take a Gaussian distribution with a covariance and generate a similar Gaussian distribution with the same number of samples and variance with a covariance. We know the cross-covariance between them and the self-covariances, so we can analytically calculate the mutual information between the two. MI is absolute which is the dependence or similarity between the two datasets. Now, we will see how the HSIC scores will do versus this variation of dataset size and shape. 

We have the following parameters:

* Samples - [500, 1K, 5K, 10K, 30K, 50K]
* Dimensions - [ 2, 3, 10, 50, 100]
* trials - `1:5`
* IT measures - Mutual Information
* Distributions - [Gaussian, T-Student]

In [9]:
# example parameters for the dataset
data_params = {}
data_params['dataset'] = ['gauss'] 
data_params['std'] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 
data_params['nu'] = [1]
data_params['trial'] = [1, 2, 3, 4, 5]
data_params['samples'] = [50, 100, 500, 1_000, 5_000]
data_params['dimensions'] = [2, 3, 10, 50, 100]

# example parameters function
example_params = DataParams()

# generates a named tuple containing the inputs and the MI 
inputs = example_params.generate_data()

## Experiment

We have a lot of parameters. So we are going to run everything in parallel so that we can save time. We will do this by giving the cartesian product of our nD list of parameters. This will give us a list of tuples where each entry is a set of parameters to evaluate. The length of this list will be the total number of parameters.

In [10]:
# create a list of all param combinations
parameters_list = list(dict_product(parameters))
data_params_list = list(dict_product(data_params))
n_params, n_data_params = len(parameters_list), len(data_params_list)
print('# of Params:', n_params, n_data_params)

# of Params: 216 1375


In [11]:
# create a list of all param combinations
parameters_list = list(dict_product(parameters))
n_params = len(parameters_list)
print('# of Params:', n_params)

# of Params: 216


In [14]:
from typing import Dict 

def step(params: Dict, data_params: Dict, inputs):




        # ====================
        # Sigma Estimator
        # ====================

        # estimate sigma
        sigma_X, sigma_Y = get_sigma(
            X=inputs.X, Y=inputs.Y, 
            method=params['sigma_estimator'][0], 
            percent=params['sigma_estimator'][1], 
            per_dimension=params['per_dimension'],
            separate_scales=params['separate_scales']
        )
        
        # ====================
        # HSIC Model
        # ====================
        # get hsic score
        score = get_hsic(
            inputs.X, inputs.Y, 
            params['scorer'], 
            sigma_X, sigma_Y
        )
        
        # ====================
        # Results
        # ====================

        # append results to dataframe
        results_df = pd.DataFrame({
            # Data Params
            'dataset': [data_params['dataset']],
            'trial': [data_params['trial']],
            'std': [data_params['std']],
            'nu': [data_params['nu']],
            'samples' : [data_params['samples']],
            'dimensions' : [data_params['dimensions']],
            # Standardize params
            'standardize': [params['standardize']],
            # Sigma type params
            'per_dimension': [params['per_dimension']],
            'separate_scales': [params['separate_scales']],
            # Gamma Params
            'sigma_method': [params['sigma_estimator'][0]],
            'sigma_percent': [params['sigma_estimator'][1]],
            'sigma_X': [sigma_X],
            'sigma_Y': [sigma_Y],
            # HSIC Params
            'scorer': [params['scorer']],
            'score': [score],
            'mutual_info': [inputs.mutual_info]
        })
        return results_df

#### Test

In [15]:
results_df = pd.DataFrame()

for iparam in data_params_list:
    print(iparam)

    # ================
    # DATA
    # ================    
    dist_data = DataParams(
        dataset=iparam['dataset'],
        trial = iparam['trial'],
        std = iparam['std'],
        nu = iparam['nu'],
        samples = iparam['samples'],
        dimensions = iparam['dimensions'],
    )
    
    # generate data
    inputs = dist_data.generate_data()
    
    # step through function


    step_df = step(parameters_list[0], iparam, inputs)
    results_df = pd.concat([results_df, step_df])
    break

{'dataset': 'gauss', 'std': 1, 'nu': 1, 'trial': 1, 'samples': 50, 'dimensions': 2}


In [16]:
results_df

,dataset,trial,std,nu,samples,dimensions,standardize,per_dimension,separate_scales,sigma_method,sigma_percent,sigma_X,sigma_Y,scorer,score,mutual_info
0,gauss,1,1,1,50,2,True,True,True,median,0.1,"[0.1686566316684468, 0.14612229488391992]","[0.1589719949193001, 0.1680410083908699]",hsic,0.019091,0.0


In [17]:
from tqdm import tqdm

In [18]:
verbose = 0
n_jobs = 16

results_df = pd.DataFrame({})


for iparam in tqdm(data_params_list, total=len(data_params_list)):
#     print(iparam)

    # ================
    # DATA
    # ================    
    dist_data = DataParams(
        dataset=iparam['dataset'],
        trial = iparam['trial'],
        std = iparam['std'],
        nu = iparam['nu'],
        samples = iparam['samples'],
        dimensions = iparam['dimensions'],
    )
    
    # generate data
    inputs = dist_data.generate_data()
    
    # step through function

    step_df = run_parallel_step(
        exp_step=step, 
        parameters=parameters_list,
        n_jobs=n_jobs,
        verbose=verbose,
        data_params=iparam,
        inputs=inputs
    )
    results_df = pd.concat([results_df, pd.concat(step_df, ignore_index=True)])
#     res_test_df
#     break

  2%|▏         | 23/1375 [14:15<64:21:20, 171.36s/it]

KeyboardInterrupt: 

In [76]:
results_df.tail()

,dataset,trial,std,nu,samples,dimensions,standardize,per_dimension,separate_scales,sigma_method,sigma_percent,sigma_X,sigma_Y,scorer,score,mutual_info
211,gauss,1,1,1,50,2,False,False,True,median,0.9,2.88461,2.80248,cka,0.044423,0.0
212,gauss,1,1,1,50,2,True,True,False,median,0.9,1.99481,1.99481,cka,0.064728,0.0
213,gauss,1,1,1,50,2,False,True,False,median,0.9,1.99481,1.99481,cka,0.064728,0.0
214,gauss,1,1,1,50,2,True,False,False,median,0.9,2.84355,2.84355,cka,0.044155,0.0
215,gauss,1,1,1,50,2,False,False,False,median,0.9,2.84355,2.84355,cka,0.044155,0.0


In [19]:
RES_PATH

'/home/emmanuel/projects/2019_hsic_align/data/results/distributions/mutual_info/'

In [23]:
with open(f'{FIG_PATH}/test.csv', 'a') as f:
    pass

In [67]:
verbose = 1
n_jobs = 16

results = run_parallel_step(
    exp_step=step, 
    parameters=parameters_list,
    n_jobs=n_jobs,
    verbose=verbose,
)

[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.


KeyboardInterrupt: 